# Text-To-Text LLM Server

This repository is designed to be used with Visual Studio Code and Docker DevContainer.

![dev-container](../img/dev-container.png)

## 1. Setup

**Instructions:**

a) Download model

```bash
huggingface-cli download bartowski/Mistral-7B-Instruct-v0.3-exl2 \
    --local-dir ~/.gai/models/exllamav2-mistral7b \
    --local-dir-use-symlinks False
```


---

## 2. Smoke Test

In [2]:
from gai.lib.server.singleton_host import SingletonHost
from gai.lib.common.utils import free_mem
from rich.console import Console
console=Console()

config = {
    "type": "ttt",
    "generator_name": "exllamav2-mistral7b",
    "engine": "gai.ttt.server.GaiExLlamaV2",
    "model_path": "models/exllamav2-mistral7b",
    "model_basename": "model",
    "max_seq_len": 8192,
    "prompt_format": "mistral",
    "hyperparameters": {
        "temperature": 0.85,
        "top_p": 0.8,
        "top_k": 50,
        "max_new_tokens": 1000,
    },
    "tool_choice": "auto",
    "max_retries": 5,
    "stop_conditions": ["<s>", "</s>", "user:","\n\n"],
    "no_flash_attn":True,
    "seed": None,
    "decode_special_tokens": False,
    "module_name": "gai.ttt.server.gai_exllamav2",
    "class_name": "GaiExLlamav2",
    "init_args": [],
    "init_kwargs": {}
}

# check .gairc
import os
gairc=None
with open(os.path.expanduser("~/.gairc"),"r") as f:
    gairc = f.read()
print(gairc)

# check ~/.gairc (if docker created .gairc)
import json
jsoned=json.loads(gairc)
assert jsoned["app_dir"]=="/home/kakkoii1337/.gai"

# check ~/.gai (if docker created the mount point)
assert os.path.exists(os.path.expanduser("~/.gai"))

# check exllamav2 (if exllamav2 is installed in the venv)
from exllamav2 import ExLlamaV2Config

# before loading
free_mem()
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:

        # after loading
        free_mem()
except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()

{"app_dir":"/home/kakkoii1337/.gai"}



Free memory: 6.17 GB

ERROR    SingletonHost: Error loading generator None: Can't find /home/kakkoii1337/.gai/models/exllamav2-mistral7b


Free memory: 6.17 GB

AssertionError: Can't find /home/kakkoii1337/.gai/models/exllamav2-mistral7b

---

## 3. Integration Test

### Startup

In [ ]:
host = SingletonHost.GetInstanceFromConfig(config, verbose=False)
host.load()
generator = host.generator
free_mem()

Free memory: 1.24 GB

1.2447967529296875

### a) Test streaming

In [ ]:
response = host.generator.create(
    messages=[{"role":"user","content":"Tell me a one paragraph story"},
                {"role":"assistant","content":""}],
    stream=True)
for message in response:
    if message.choices[0].delta.content:
        print(message.choices[0].delta.content, end="", flush=True)
   

 Once upon a time, in a small village nestled between the mountains, lived a kind-hearted and hardworking farmer named Tomas. Despite the harsh conditions, he worked tirelessly to sustain his family and help his neighbors. One day, a great storm swept through the valley, destroying Tomas's crops and leaving him with nothing. However, the villagers rallied together, sharing their own resources to help Tomas rebuild. The storm may have taken his crops, but it couldn't break the spirit of the community that stood by him.


### b) Test generation

In [ ]:

response = host.generator.create(
    messages=[{"role":"user","content":"Tell me a one paragraph story"},
                {"role":"assistant","content":""}],
    stream=False)
print(response.choices[0].message.content)


 Once upon a time, in a small village nestled between the mountains, lived a kind-hearted and resourceful young woman named Mia. She was known for her healing herbs and warm smile. One day, a traveling merchant lost his way in a storm and sought shelter in Mia's humble abode. Grateful for her hospitality, he left behind a magical gem, which granted Mia the power to heal any ailment. This gem transformed her into a renowned healer, and the villagers lived in peace and prosperity, thanks to Mia's selfless actions and the magical gem's power.


### c) Test Tool Calling

In [ ]:
messages = [
    {"role":"user","content":"What is the current time in Singapore?"},
    {"role":"assistant","content":""}
]
tool_choice="required"
tools = [
    {
        "type": "function",
        "function": {
            "name": "google",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    }
]
response = host.generator.create(
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    stream=False)
print(response)


ChatCompletion(id='chatcmpl-b58099e8-b79c-466d-9956-aafb3242b648', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fc9d1739-2842-43d0-983b-8e07961dd8fc', function=Function(arguments='{"search_query": "current time Singapore"}', name='google'), type='function')]))], created=1723901723, model='exllamav2-mistral7b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=336, total_tokens=357))


### d) Test Structured Output

In [ ]:
# Define Schema
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int

text = """Foundation is a science fiction novel by American writer
Isaac Asimov. It is the first published in his Foundation Trilogy (later
expanded into the Foundation series). Foundation is a cycle of five
interrelated short stories, first published as a single book by Gnome Press
in 1951. Collectively they tell the early story of the Foundation,
an institute founded by psychohistorian Hari Seldon to preserve the best
of galactic civilization after the collapse of the Galactic Empire.
"""
response = host.generator.create(messages=[{'role':'user','content':text},{'role':'assistant','content':''}], 
    json_schema=Book.schema(),
    stream=False
    )
print(response)


ChatCompletion(id='chatcmpl-536f4c2b-b4aa-4a19-a20b-6477bf652d7f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=' {\n  "title": "Foundation",\n  "summary": "Foundation is a science fiction novel by Isaac Asimov, the first published in his Foundation Trilogy. It is a cycle of five interrelated short stories that tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire.",\n  "author": "Isaac Asimov",\n  "published_year": 1951\n}', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723901730, model='exllamav2-mistral7b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=116, prompt_tokens=257, total_tokens=373))


### Teardown

In [ ]:
del host.generator.model
del host.generator.cache
del host.generator.tokenizer
del host.generator
import gc,torch
gc.collect()
torch.cuda.empty_cache()
free_mem()

Free memory: 1.42 GB

1.4186210632324219

---

## 4. API Test

**Instructions**:

a) Press `F5` to start the API server.

**Tests**:

Run the following cells to test the API.

### a) Test Generating

In [ ]:
%%bash
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"exllamav2-mistral7b\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Tell me a story.\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"tool_choice\": \"none\"}"
        

{"id":"chatcmpl-4c565ddc-ee01-4b63-bec5-fb8044de5075","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":" Once upon a time, in a land far, far away, there was a kingdom surrounded by a dense forest. The kingdom was ruled by a wise and just king, who loved his people dearly. However, the kingdom was plagued by a fearsome beast that lived deep within the forest. The beast would often come out at night and terrorize the villagers, causing them to live in constant fear","refusal":null,"role":"assistant","function_call":null,"tool_calls":null}}],"created":1724600640,"model":"exllamav2-mistral7b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":86,"prompt_tokens":14,"total_tokens":100}}

### b) Test Streaming

In [ ]:
import json
import httpx

# Generate the JSON payload
json_payload = {
    "temperature": 0.2,
    "max_new_tokens": 1000,
    "stream": "true",
    "messages": [
        {
            "role": "user",
            "content": "Tell me a one paragraph story."
        },
        {
            "role": "assistant",
            "content": ""
        }
    ]
}

# Send the POST request using httpx with streaming
with httpx.Client() as client:
    response = client.post("http://localhost:12031/gen/v1/chat/completions", json=json_payload)
    for line in response.iter_lines():
        result = json.loads(line)
        content = result["choices"][0]["delta"]["content"]
        if content:
            print(content, end="", flush=True)


 Once upon a time, in a small village nestled between the mountains, lived a young girl named Mia. She was known for her kindness and her love for animals. One day, a wounded bird fell from the sky, unable to fly. Mia took it in, nursed it back to health, and released it back into the wild when it was better. This act of kindness sparked a movement in the village, inspiring others to help animals in need, and Mia became a beacon of compassion in her community

### c) Test Tool Calling

In [ ]:
%%bash
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"exllamav2-mistral7b\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"What is the current time in Singapore\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"tools\": [\
            {\
                \"type\": \"function\",\
                \"function\": {\
                    \"name\": \"google\",\
                    \"description\": \"The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.\",\
                    \"parameters\": {\
                        \"type\": \"object\",\
                        \"properties\": {\
                            \"search_query\": {\
                                \"type\": \"string\",\
                                \"description\": \"The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively.\"\
                            }\
                        },\
                        \"required\": [\"search_query\"]\
                    }\
                }\
            }\
        ],\
        \"tool_choice\": \"required\"}"

{"id":"chatcmpl-45ce3df8-8340-40fb-8e84-eed89453daa7","choices":[{"finish_reason":"tool_calls","index":0,"logprobs":null,"message":{"content":null,"refusal":null,"role":"assistant","function_call":null,"tool_calls":[{"id":"call_a67bf2e6-ff7c-42c0-9d70-e44b82eb167c","function":{"arguments":"{\"search_query\": \"current time in Singapore\"}","name":"google"},"type":"function"}]}}],"created":1724600717,"model":"exllamav2-mistral7b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":22,"prompt_tokens":335,"total_tokens":357}}

### d) Test JSON Schema

In [ ]:
%%bash
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"exllamav2-mistral7b\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Foundation is a science fiction novel by American writer \
            Isaac Asimov. It is the first published in his Foundation Trilogy (later \
            expanded into the Foundation series). Foundation is a cycle of five \
            interrelated short stories, first published as a single book by Gnome Press \
            in 1951. Collectively they tell the early story of the Foundation, \
            an institute founded by psychohistorian Hari Seldon to preserve the best \
            of galactic civilization after the collapse of the Galactic Empire.\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"json_schema\": {\"properties\": \
            {\"title\": \
                {\"title\": \"Title\", \"type\": \"string\"}, \
                    \"summary\": {\"title\": \"Summary\", \"type\": \"string\"}, \
                    \"author\": {\"title\": \"Author\", \
                    \"type\": \"string\"\
                }, \
                \"published_year\": {\
                    \"title\": \"Published Year\", \
                    \"type\": \"integer\"}}, \
                \"required\": [\
                    \"title\", \
                    \"summary\", \
                    \"author\", \
                    \"published_year\"\
                ], \
                \"title\": \"Book\", \
                \"type\": \"object\"\
            },\
        \"tool_choice\": \"none\"}"

{"id":"chatcmpl-8fff830c-45de-4e00-bca1-72289622ed2b","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":" {\n  \"title\": \"Foundation\",\n  \"summary\": \"Foundation is a science fiction novel by Isaac Asimov, the first published in his Foundation Trilogy. It is a cycle of five interrelated short stories that tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire.\",\n  \"author\": \"Isaac Asimov\",\n  \"published_year\": 1951\n}","refusal":null,"role":"assistant","function_call":null,"tool_calls":null}}],"created":1724208911,"model":"exllamav2-mistral7b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":116,"prompt_tokens":253,"total_tokens":369}}

### e) Shut down the API Service

---

## 5. Docker

This test should **NOT** be run in devcontainer.

**Instructions:** 

- Click on bottom left blue button and select **Reopen Folder in WSL**

- Create and activate conda environment

    ```bash
    conda env create -f environment.yml
    conda activate gai-ttt
    ```

- Press **CTRL+SHIFT+P** > **Python: Select Interpreter**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **Docker: build**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **Docker: run**

**Tests:**

Repeat the API test (#)

**Tear Down:**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **Docker: stop**